### Imports
### Make sure you add to this cell as you experiement with other types of RNN

In [16]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import keras
import keras.utils
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from sklearn.metrics import confusion_matrix

### This section deals with loading the training and testing datasets
### it is not necessary to modify it for the PE10 assignment

In [17]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames):
	loaded = list()
	for name in filenames:
		data = load_file(name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_train():
	filenames = ['total_acc_x_train.txt','total_acc_y_train.txt','total_acc_z_train.txt','body_acc_x_train.txt','body_acc_y_train.txt','body_acc_z_train.txt','body_gyro_x_train.txt','body_gyro_y_train.txt','body_gyro_z_train.txt']
	# load input data
	X = load_group(filenames)
	# load class output
	y = load_file('y_train.txt')
	return X, y

def load_dataset_test():
  filenames = ['total_acc_x_test.txt','total_acc_y_test.txt','total_acc_z_test.txt','body_acc_x_test.txt','body_acc_y_test.txt','body_acc_z_test.txt','body_gyro_x_test.txt','body_gyro_y_test.txt','body_gyro_z_test.txt']
  # load input data
  X = load_group(filenames)
  # load class output
  y = load_file('y_test.txt')
  return X, y


# load the dataset, returns train and test X and y elements
def load_dataset():
	# load all train
	trainX, trainy = load_dataset_train()
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_test()
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [18]:
# Function to build, compile, and train the model
# For PE10, copy and modify this section to try different model designs and RNNs such as simpleRNN and GRU, in addition to LSTM
# You can try stacking the RNN layers. Refer to the Keras API on how to do this

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 15, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  model = Sequential()
  model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  #model.summary()
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy


### This function summarizes the results of the training

In [19]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

### This is the main code that loads the data and runs the classifier

In [20]:
# run an experiment
def run_experiment(repeats=10):
  # load data
  trainX, trainy, testX, testy = load_dataset()
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

### Running the code here:
### Make sure you go back and modify the model design when trying out new versions for simpleRNN, LSTM and GRU. 

In [21]:
# run the experiment
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 90.601
>#2: 91.686
>#3: 89.040
>#4: 91.042
>#5: 89.888
>#6: 90.499
>#7: 89.074
>#8: 90.261
>#9: 90.329
>#10: 89.175
[90.60060977935791, 91.68646335601807, 89.03970122337341, 91.0417377948761, 89.88802433013916, 90.49881100654602, 89.07363414764404, 90.26128053665161, 90.32914638519287, 89.17543292045593]
Accuracy: 90.159% (+/-0.833)


In [23]:
from keras.layers import ConvLSTM2D
#Develop a ConvLSTM Network Model


def evaluate_model2(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 0, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape into subsequences (samples, time steps, rows, cols, channels)
	n_steps, n_length = 4, 32
	trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
	# define model
	model2 = Sequential()
	model2.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
	model2.add(Dropout(0.5))
	model2.add(Flatten())
	model2.add(Dense(100, activation='relu'))
	model2.add(Dense(n_outputs, activation='softmax'))
	model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model2.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model2.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy


In [24]:
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [25]:
# run an experiment
def run_experiment(repeats=5):
  # load data
  trainX, trainy, testX, testy = load_dataset()
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model2(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

In [26]:
# run the experiment
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 90.329
>#2: 92.162
>#3: 91.177
>#4: 91.245
>#5: 89.243
[90.32914638519287, 92.16151833534241, 91.17746949195862, 91.24533534049988, 89.24329876899719]
Accuracy: 90.831% (+/-0.983)


In [43]:


from keras.layers import SimpleRNN
#Develop a SimpleRNN


def evaluate_model3(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 15, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  model3 = Sequential()
  model3.add(SimpleRNN(100, input_shape=(n_timesteps,n_features)))
  model3.add(Dropout(0.5))
  model3.add(Dense(100, activation='relu'))
  model3.add(Dense(n_outputs, activation='softmax'))
  #model.summary()
  model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model3.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  _, accuracy = model3.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy


In [44]:
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [45]:
# run an experiment
def run_experiment(repeats=5):
  # load data
  trainX, trainy, testX, testy = load_dataset()
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model3(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

In [46]:
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 71.021
>#2: 70.750
>#3: 57.279
>#4: 85.918
>#5: 64.201
[71.02137804031372, 70.74991464614868, 57.27858543395996, 85.91788411140442, 64.20088410377502]
Accuracy: 69.834% (+/-9.490)


In [55]:
# Develop a GRU model
from keras.layers import GRU

def evaluate_model3(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 15, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  model4 = Sequential()
  model4.add(GRU(100, input_shape=(n_timesteps,n_features)))
  model4.add(Dropout(0.5))
  model4.add(Dense(100, activation='relu'))
  model4.add(Dense(n_outputs, activation='softmax'))
  #model.summary()
  model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model4.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  _, accuracy = model4.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy


In [56]:
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [57]:
# run an experiment
def run_experiment(repeats=5):
  # load data
  trainX, trainy, testX, testy = load_dataset()
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model4(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

In [58]:
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 73.057
>#2: 80.014
>#3: 82.321
>#4: 70.445
>#5: 53.885
[73.05734753608704, 80.01357316970825, 82.3210060596466, 70.44451832771301, 53.88530492782593]
Accuracy: 71.944% (+/-10.024)
